In [1]:
#import libraries
import os
import time
import csv
import pandas as pd
import numpy as np
import bs4 as bs
from itertools import product
from urllib.request import Request
import urllib
import sys
import json
import PyPDF2
import collections
import nltk
from collections import Counter
import math
import string
import textmining as txtm


#working directory path
path = 'C:/Users/Lenovo/Desktop/NLP/'
#path = 'C:/Users/user/Python/Natural Language Processsing/'
os.chdir(path)

def download_file(download_url,fname='document.pdf'):
    if sys.version_info[0]==2:
        response = urllib.urlopen(download_url) # python 2
    else:
        response = urllib.request.urlopen(download_url) # python 3

    file = open(fname, 'wb')
    file.write(response.read())
    file.close()
    
#check if the html files contain the word 'privacy' 
#restrict documents that have length less than 100 words
def check_privacy(firm,url,text,min_words=100,domain = ['.com','.gov']):
    if domain[0] not in url:
    #if (domain[0] not in url and domain[1] not in url): 
        flag = "Error:  in url %s corresponding to %s No %s domain" % (url,firm,domain)
    elif ('privacy' not in text.lower()):
        flag = "Error:  in url %s corresponding to %s Does not contain the word privacy" % (url , firm )
    elif len(str(text).split()) < min_words:
        flag = "Error : in url %s corresponding to %s Less than %s words" %  (url , firm ,str(min_words)) 
    else:
        flag = "url %s corresponding to %s is _OK_" % (url,firm)
    return flag

In [2]:
# import stopwords from R's 'tm' library
# import state names of US

rsw = pd.read_csv('C:/Users/Lenovo/Desktop/NLP/rstopwords.csv')['x'].values.tolist() 

states = pd.read_csv('C:/Users/Lenovo/Desktop/NLP/states.csv')['State'].values.tolist()

In [3]:
# load urls
with open("C:/Users/Lenovo/Desktop/NLP/compustat_trial__ggl_urls.json",'r') as fp:
    urls = json.load(fp)
   
firms = list(urls.keys())


In [4]:
len(firms)

144

In [5]:
firms = list(urls.keys())[0:105:1]

In [6]:
# create master dictionary 
scrape_policies = {}
scrape_errors = {}
replace_strings = ["\\xcb\\x9","\\xc5\\x","\\xef\\xac\\x81","\\xef\\xac\\x82","\b",
                   "\\xe2\\x84\\xa2s","\\\\xe2\\\\x84\\\\xa2s","\\', b\\'",
                   "\\\\xc5\\\\xa0","\\\\xe2\\\\x84\\\\xa2","\\xe2\\x84\\xa2"]


# access the text extracted for each firm using indexes
# change in Load_urls tab from the next line to run for all firms.
for firm in firms:
    scrape_policies[firm] = {}
    scrape_errors[firm] = {}

    for index in [0,1,2,3,4]:
        policies = ""
        length = 0
        try:
            path = 'C:/Users/Lenovo/Desktop/NLP/'
            #path = 'C:/Users/user/Python/Natural Language Processsing/'
            os.chdir(path)
            if urls[firm][index].endswith('.pdf'):
                os.chdir('PDF')
                #check if the file is pdf
                fname="%s_%s.pdf" % (firm,str(index))
                download_file(urls[firm][index], fname)
                
                #extract texts from the pdfs
                pdf_file = open(fname, 'rb')
                read_pdf = PyPDF2.PdfFileReader(pdf_file)
                number_of_pages = read_pdf.getNumPages()
                c = collections.Counter(range(number_of_pages))
                content = ""
                for i in c:
                    page = read_pdf.getPage(i)
                    page_content = page.extractText()
                    content = content + ' ' + str(page_content.encode('utf-8'))
                for string in replace_strings:
                    content = content.replace(string,'')   
                
                
                flag = 'Downloaded %s chars from %s document' % (len(str(content)),fname)
                policies = content
            else:
                sauce = urllib.request.urlopen(Request(urls[firm][index], headers={'User-Agent': 'Mozilla/5.0'})).read()
                soup = bs.BeautifulSoup(sauce,'html5lib')
                for i in list(range(1,len(soup.find_all(['p','ul','table'])))):
                    text = soup.find_all(['p','ul','table'])[i].text
                    policies = policies + ' ' + text
                    length= length+len(text)
                flag = "Downloaded %d chars" % length
            
        except Exception as e:
            flag = "Error: " + str(e)
            policies = flag
        finally:
            print('Firm: %s, URL %s, %s' % (firm,str(index),flag))
        if len(policies) != 0:
            scrape_policies[firm][index] = policies
        else:
            #'a an the' has been used only for indexing purpose - will any way be removed for not meeting 100 words criteria
            scrape_policies[firm][index] = 'a an the'
    

Firm: STANDARD MOTOR PRODUCTS , URL 0, Downloaded 3149 chars
Firm: STANDARD MOTOR PRODUCTS , URL 1, Downloaded 10865 chars
Firm: STANDARD MOTOR PRODUCTS , URL 2, Downloaded 5220 chars
Firm: STANDARD MOTOR PRODUCTS , URL 3, Error: HTTP Error 406: Not Acceptable
Firm: STANDARD MOTOR PRODUCTS , URL 4, Downloaded 1514 chars
Firm: STANDEX INTERNATIONAL , URL 0, Downloaded 17945 chars
Firm: STANDEX INTERNATIONAL , URL 1, Downloaded 1819 chars
Firm: STANDEX INTERNATIONAL , URL 2, Downloaded 35127 chars
Firm: STANDEX INTERNATIONAL , URL 3, Downloaded 13102 chars
Firm: STANDEX INTERNATIONAL , URL 4, Downloaded 49848 chars
Firm: BAYER AG , URL 0, Error: 'NoneType' object has no attribute 'next_element'
Firm: BAYER AG , URL 1, Downloaded 18233 chars
Firm: BAYER AG , URL 2, Downloaded 13248 chars
Firm: BAYER AG , URL 3, Downloaded 21718 chars
Firm: BAYER AG , URL 4, Downloaded 3908 chars
Firm: DEUTSCHE LUFTHANSA AG , URL 0, Error: HTTP Error 405: Method Not Allowed
Firm: DEUTSCHE LUFTHANSA AG , UR

Firm: CECO ENVIRONMENTAL , URL 0, Downloaded 59014 chars
Firm: CECO ENVIRONMENTAL , URL 1, Downloaded 36407 chars
Firm: CECO ENVIRONMENTAL , URL 2, Downloaded 43517 chars
Firm: CECO ENVIRONMENTAL , URL 3, Downloaded 34650 chars
Firm: CECO ENVIRONMENTAL , URL 4, Downloaded 42910 chars
Firm: CEMEX SAB DE CV , URL 0, Downloaded 24567 chars
Firm: CEMEX SAB DE CV , URL 1, Downloaded 43397 chars
Firm: CEMEX SAB DE CV , URL 2, Downloaded 17969 chars
Firm: CEMEX SAB DE CV , URL 3, Downloaded 47641 chars
Firm: CEMEX SAB DE CV , URL 4, Downloaded 44086 chars
Firm: TEXTRON , URL 0, Downloaded 9983 chars
Firm: TEXTRON , URL 1, Downloaded 28998 chars
Firm: TEXTRON , URL 2, Downloaded 8912 chars
Firm: TEXTRON , URL 3, Downloaded 3005 chars
Firm: TEXTRON , URL 4, Downloaded 5244 chars
Firm: ARC RESOURCES , URL 0, Downloaded 196 chars
Firm: ARC RESOURCES , URL 1, Downloaded 953 chars
Firm: ARC RESOURCES , URL 2, Downloaded 5593 chars
Firm: ARC RESOURCES , URL 3, Downloaded 0 chars
Firm: ARC RESOURCES 

Firm: INSPIRATION MINING , URL 0, Downloaded 4364 chars
Firm: INSPIRATION MINING , URL 1, Downloaded 7737 chars
Firm: INSPIRATION MINING , URL 2, Downloaded 3732 chars
Firm: INSPIRATION MINING , URL 3, Downloaded 12130 chars
Firm: INSPIRATION MINING , URL 4, Downloaded 3770 chars
Firm: KELSO TECHNOLOGIES , URL 0, Downloaded 7401 chars
Firm: KELSO TECHNOLOGIES , URL 1, Error: HTTP Error 999: Request denied
Firm: KELSO TECHNOLOGIES , URL 2, Downloaded 2407 chars
Firm: KELSO TECHNOLOGIES , URL 3, Downloaded 3010 chars
Firm: KELSO TECHNOLOGIES , URL 4, Error: HTTP Error 403: Forbidden
Firm: TRIBUNE MEDIA , URL 0, Downloaded 18664 chars
Firm: TRIBUNE MEDIA , URL 1, Downloaded 86659 chars
Firm: TRIBUNE MEDIA , URL 2, Downloaded 4632 chars
Firm: TRIBUNE MEDIA , URL 3, Downloaded 19815 chars
Firm: TRIBUNE MEDIA , URL 4, Downloaded 2577 chars
Firm: MACATAWA BANK , URL 0, Downloaded 74812 chars
Firm: MACATAWA BANK , URL 1, Downloaded 4330 chars
Firm: MACATAWA BANK , URL 2, Downloaded 68731 chars

Firm: ETRION , URL 2, Error: HTTP Error 999: Request denied
Firm: ETRION , URL 3, Downloaded 6716 chars
Firm: ETRION , URL 4, Downloaded 5085 chars
Firm: EVOLVING SYSTEMS , URL 0, Downloaded 17071 chars
Firm: EVOLVING SYSTEMS , URL 1, Downloaded 27008 chars
Firm: EVOLVING SYSTEMS , URL 2, Downloaded 3149 chars
Firm: EVOLVING SYSTEMS , URL 3, Downloaded 4558 chars
Firm: EVOLVING SYSTEMS , URL 4, Downloaded 28231 chars
Firm: FIRST GUARANTY BANCSHARES , URL 0, Downloaded 2245 chars
Firm: FIRST GUARANTY BANCSHARES , URL 1, Downloaded 23573 chars
Firm: FIRST GUARANTY BANCSHARES , URL 2, Downloaded 3912 chars
Firm: FIRST GUARANTY BANCSHARES , URL 3, Downloaded 2971 chars
Firm: FIRST GUARANTY BANCSHARES , URL 4, Downloaded 5766 chars
Firm: HACKETT GROUP , URL 0, Downloaded 28923 chars
Firm: HACKETT GROUP , URL 1, Downloaded 0 chars
Firm: HACKETT GROUP , URL 2, Downloaded 8906 chars
Firm: HACKETT GROUP , URL 3, Downloaded 974 chars
Firm: HACKETT GROUP , URL 4, Downloaded 7738 chars
Firm: HAWTH

In [7]:
#remove policies that do not contain 100 words (minimum length)
# Also checking whether the text contains 'privacy' or not using the check_privacy function.
 
okurl = {}
for i in firms:
    okurl[i] = {}

for firm in firms:
    position = []
    for index in range(0,5):
        temp = (check_privacy(firm = firm , url = urls[firm][index] , text = scrape_policies[firm][index]))
        if '_OK_' in temp:
            position.append(index)
            print('%s index %s' % (temp , index) )
        okurl[firm] = position
        

url https://www.smpcorp.com/en/privacy/ corresponding to STANDARD MOTOR PRODUCTS  is _OK_ index 0
url http://www.smpcorp.com/en/ corresponding to STANDARD MOTOR PRODUCTS  is _OK_ index 1
url https://careers.smpcorp.com/search corresponding to STANDARD MOTOR PRODUCTS  is _OK_ index 2
url https://careers.smpcorp.com/en-US/page/interns corresponding to STANDARD MOTOR PRODUCTS  is _OK_ index 4
url https://standexelectronics.com/privacy-policy/ corresponding to STANDEX INTERNATIONAL  is _OK_ index 0
url http://www.standex.com/about/contact-standex corresponding to STANDEX INTERNATIONAL  is _OK_ index 1
url http://www.standexparts.com/privacy-standex corresponding to STANDEX INTERNATIONAL  is _OK_ index 2
url http://www.standexengraving.com/privacy-policy corresponding to STANDEX INTERNATIONAL  is _OK_ index 3
url https://innovent.com/privacy-policy.htm corresponding to STANDEX INTERNATIONAL  is _OK_ index 4
url http://pharma.bayer.com/en/privacy-statement/ corresponding to BAYER AG  is _OK_

url https://www.prnewswire.com/news-releases/conrad-industries-announces-first-quarter-2018-results-and-backlog-300649110.html corresponding to CONRAD INDUSTRIES  is _OK_ index 3
url http://www.crai.com/privacy corresponding to CRA INTERNATIONAL  is _OK_ index 0
url https://www.zacks.com/stock/quote/CRAI corresponding to CRA INTERNATIONAL  is _OK_ index 2
url https://orderussco.com/Website/index.jsp?rspage=/PublicSite/tandc.htm corresponding to ESSENDANT  is _OK_ index 1
url http://investors.essendant.com/CustomPage/Index?KeyGenPage=210135 corresponding to ESSENDANT  is _OK_ index 3
url https://www.azerty.com/Website/index.jsp?rspage=/PublicSite/tandc.htm corresponding to ESSENDANT  is _OK_ index 4
url https://www.etrion.com/en/terms-of-use/ corresponding to ETRION  is _OK_ index 0
url https://www.pv-magazine.com/2018/05/07/etrion-halves-net-loss-in-q1/ corresponding to ETRION  is _OK_ index 1
url https://www.pv-magazine.com/2016/12/13/etrion-sells-53-4-mw-of-solar-portfolio-to-ef-sola

In [8]:
okurl

{'STANDARD MOTOR PRODUCTS ': [0, 1, 2, 4],
 'STANDEX INTERNATIONAL ': [0, 1, 2, 3, 4],
 'BAYER AG ': [2, 3, 4],
 'DEUTSCHE LUFTHANSA AG ': [1, 3],
 'AMCOR ': [1],
 'STATE STREET ': [0, 2, 3, 4],
 'STEPAN ': [0, 2, 3, 4],
 'VOLKSWAGEN AG ': [0, 1, 2, 3, 4],
 'NIELSEN HOLDINGS ': [0, 1, 2, 3, 4],
 'PUBLIC STORAGE ': [0, 1, 2, 4],
 'STRYKER ': [0, 1, 3, 4],
 'NAVIENT ': [0, 2, 4],
 'STURM RUGER ': [0, 1, 3],
 'NOVARTIS AG ': [0, 2, 3, 4],
 'SUNTRUST BANKS ': [0, 1, 2, 3, 4],
 'SUPERIOR INDUSTRIES ': [0, 4],
 'SYNALLOY ': [0, 1, 2, 3, 4],
 'SAPPI ': [1, 3, 4],
 'SYSCO ': [0, 2, 3],
 'CGG ': [0, 1, 2, 3, 4],
 'SAP SE ': [0, 1, 3],
 'TEJON RANCH ': [1, 2, 3, 4],
 'TELEFLEX ': [0, 3, 4],
 'TENNECO ': [0, 2, 3, 4],
 'TENNESSEE VALLEY AUTHORITY ': [0, 1, 2, 3],
 'AMERICAN AIRLINES GROUP ': [0, 2],
 'TERADYNE ': [0, 2, 3, 4],
 'UNITED AIRLINES ': [3],
 'TEXAS INSTRUMENTS ': [0, 2, 4],
 'CECO ENVIRONMENTAL ': [0, 1, 2, 3, 4],
 'CEMEX SAB DE CV ': [0, 1, 2, 3, 4],
 'TEXTRON ': [0, 1, 2, 3, 4],
 'A

In [9]:
#compare texts extracted from both html files and pdfs
#select the one that has the longest length from the list (pdf & html included)
def testFuncNew(text= 'This is the sample text.',sw=rsw):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in sw])
    return text

for firm in firms:
    length = []
    if len(okurl[firm]) != 0:
        #for i in okurl[firm]:
            #length.append(len(testFuncNew(scrape_policies[firm][i])))

        #maxvalue = max(length)
        okurl[firm] = [okurl[firm][0]]
    if len(okurl[firm]) == 0:
        del okurl[firm]

In [10]:
okurl

{'STANDARD MOTOR PRODUCTS ': [0],
 'STANDEX INTERNATIONAL ': [0],
 'BAYER AG ': [2],
 'DEUTSCHE LUFTHANSA AG ': [1],
 'AMCOR ': [1],
 'STATE STREET ': [0],
 'STEPAN ': [0],
 'VOLKSWAGEN AG ': [0],
 'NIELSEN HOLDINGS ': [0],
 'PUBLIC STORAGE ': [0],
 'STRYKER ': [0],
 'NAVIENT ': [0],
 'STURM RUGER ': [0],
 'NOVARTIS AG ': [0],
 'SUNTRUST BANKS ': [0],
 'SUPERIOR INDUSTRIES ': [0],
 'SYNALLOY ': [0],
 'SAPPI ': [1],
 'SYSCO ': [0],
 'CGG ': [0],
 'SAP SE ': [0],
 'TEJON RANCH ': [1],
 'TELEFLEX ': [0],
 'TENNECO ': [0],
 'TENNESSEE VALLEY AUTHORITY ': [0],
 'AMERICAN AIRLINES GROUP ': [0],
 'TERADYNE ': [0],
 'UNITED AIRLINES ': [3],
 'TEXAS INSTRUMENTS ': [0],
 'CECO ENVIRONMENTAL ': [0],
 'CEMEX SAB DE CV ': [0],
 'TEXTRON ': [0],
 'ARC RESOURCES ': [2],
 'THERMO FISHER SCIENTIFIC ': [0],
 'JAEGER RESOURCES ': [2],
 'AMERICAN TOWER ': [0],
 'THOR INDUSTRIES ': [0],
 'BADGER DAYLIGHTING ': [0],
 'TIDEWATER ': [0],
 'SIX FLAGS ENTERTAINMENT ': [0],
 'AFRICA OIL ': [0],
 'TIMKEN ': [0],


In [11]:
firms = list(okurl.keys())

In [12]:
#function to be used to extract headers 
def testFuncNew(text= 'This is sample text.',sw=rsw,key = 'e'):
    if key == 'e':
        text = text.lower()
        text = ' '.join([word for word in text.split() if word not in sw])
    else:
        text = text.lower()
        text = ' '.join([word for word in text.split()])
    return text

In [13]:
#extract and store header of paragraphs
# Dictionary for storing and accessing headers.

Header_policies = []
Paragraph_policies= []
Headers_Lengths = []

#function to be used to extract headers 
def testFuncNew(text= 'This is sample text.',sw=rsw,key = 'e'):
    if key == 'e':
        text = text.lower()
        text = ' '.join([word for word in text.split() if word not in sw])
    else:
        text = text.lower()
        text = ' '.join([word for word in text.split()])
    return text


#loop over the five urls
for firm in firms:
    if len(okurl[firm]) == 0:
        Header_policies.append(['__NO__ __OK__ __URL__'])
        Paragraph_policies.append(['__NO__ __OK__ __URL__'])
        print('%s __NO__ __OK__ __URL__' %firm)
        Headers_Lengths.append(0)

    for index in okurl[firm]:
        policies_h = []
        policies_p = []
        try:                
            if urls[firm][index].endswith('.pdf'):
                flag = 'Document is PDF'
                policies_h.append('Document is PDF')
                policies_p.append('Document is PDF')
                Headers_Lengths.append(0)
            else:
                sauce = urllib.request.urlopen(Request(urls[firm][index], headers={'User-Agent': 'Mozilla/5.0'})).read()
                #read the html files using bs - use 'lxml' is 'html5lib' doesn't work
                soup = bs.BeautifulSoup(sauce,'html5lib')
                headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5','h6'])
                paras = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5','h6','p','ul','table'])
                headers = [header.get_text() for header in headers]
                paras = [para.get_text() for para in paras]
                
                headers = [x for x in headers if x != '']
                #extract headers - isolate bold texts from the main text
                for header_index,header_word in enumerate(headers):
                    headers[header_index] = testFuncNew(text = header_word,key='ne').upper()
                policies_h = headers
                for para_index,para_sentence in enumerate(paras):
                    paras[para_index] = testFuncNew(text = para_sentence,key='ne').lower()
                policies_p = paras
                flag = "Downloaded %d headers" % len(headers)
                Headers_Lengths.append(len(headers))
            
        except Exception as e:
            flag = "Error: " + str(e)
            policies_h.append(flag)
            policies_p.append(flag)
            Headers_Lengths.append(0)
        finally:
            print('Firm: %s, URL %s, %s' % (firm,str(index),flag))
            Header_policies.append(policies_h)
            Paragraph_policies.append(policies_p)

Firm: STANDARD MOTOR PRODUCTS , URL 0, Downloaded 1 headers
Firm: STANDEX INTERNATIONAL , URL 0, Downloaded 19 headers
Firm: BAYER AG , URL 2, Downloaded 21 headers
Firm: DEUTSCHE LUFTHANSA AG , URL 1, Downloaded 32 headers
Firm: AMCOR , URL 1, Downloaded 4 headers
Firm: STATE STREET , URL 0, Downloaded 18 headers
Firm: STEPAN , URL 0, Downloaded 7 headers
Firm: VOLKSWAGEN AG , URL 0, Downloaded 10 headers
Firm: NIELSEN HOLDINGS , URL 0, Downloaded 15 headers
Firm: PUBLIC STORAGE , URL 0, Downloaded 0 headers
Firm: STRYKER , URL 0, Downloaded 2 headers
Firm: NAVIENT , URL 0, Downloaded 1 headers
Firm: STURM RUGER , URL 0, Downloaded 2 headers
Firm: NOVARTIS AG , URL 0, Downloaded 12 headers
Firm: SUNTRUST BANKS , URL 0, Downloaded 66 headers
Firm: SUPERIOR INDUSTRIES , URL 0, Downloaded 22 headers
Firm: SYNALLOY , URL 0, Downloaded 6 headers
Firm: SAPPI , URL 1, Downloaded 2 headers
Firm: SYSCO , URL 0, Downloaded 6 headers
Firm: CGG , URL 0, Downloaded 14 headers
Firm: SAP SE , URL 0,

In [14]:
# Eleminating blank headers 
for index,header in enumerate(Header_policies):
    temp = [x for x in header if x != '']
    Header_policies[index] = temp
    

In [15]:
Header_policies

[['PRIVACY'],
 ['PRIVACY POLICY',
  'INTRODUCTION',
  'WHAT IS PERSONAL DATA?',
  'THE INFORMATION WE COLLECT',
  'HOW WE USE THE INFORMATION WE COLLECT',
  'TO WHOM WE DISCLOSE INFORMATION AND WHY',
  'HOW TO ACCESS AND CONTROL THE INFORMATION',
  'DATA SECURITY AND STORAGE',
  'DATA RETENTION',
  'OUR POLICY TOWARDS CHILDREN',
  'CALIFORNIA PRIVACY RIGHTS',
  'CHANGES TO OUR PRIVACY POLICY',
  'TERMS OF USE',
  'CONTACT INFORMATION',
  'SUBSCRIBE TO OUR NEWSLETTER',
  'NEWS & ANNOUNCEMENTS',
  'TRADESHOWS & EVENTS',
  'PRIVACY OVERVIEW',
  'STRICTLY NECESSARY COOKIES'],
 ['GLOBAL WEBSITE, PHARMACEUTICALS DIVISION, BAYER AG',
  'DATA PRIVACY STATEMENT',
  'HANDLING OF PERSONAL DATA',
  'USING OUR WEBSITE',
  'ACCESSING OUR WEBSITE',
  'SETTING OF COOKIES',
  'WHAT ARE COOKIES?',
  'WHAT COOKIES DO WE USE?',
  'OPTIONAL COOKIES',
  'SUBJECT TO YOUR CONSENT',
  'HOW TO PREVENT THE SETTING OF COOKIES',
  'WEBSITE ANALYSIS',
  'USE OF CONTACT FORMS',
  'SUBSCRIPTION TO OUR NEWSLETTER',
  

In [16]:
paragraphs = []
headers_paragraphs = []
Firm_Info = []


for firm,firm_name in list(enumerate(firms)):
    alltext = Paragraph_policies[firm]
    headers = Header_policies[firm]
    temp1 = []
    
    n_headers = len(headers)
    n_alltext = len(alltext)
    
    if n_headers == 1:
        paragraphs.append(' '.join(str(e) for e in alltext))
        temp1.append(' '.join(str(e) for e in alltext))
        Firm_Info.append(firm_name)
        
    else:
        for head in list(range(1,n_headers)):
            check1 = headers[head-1]
            check2 = headers[head]
            pos1 = alltext.index(check1.lower())
            pos2 = alltext.index(check2.lower())
            if (pos2 - pos1) > 1:
                paragraphs.append(' '.join(str(e) for e in alltext[(pos1+1):(pos2)]))
                temp1.append(' '.join(str(e) for e in alltext[(pos1+1):(pos2)]))
                Firm_Info.append(firm_name)
            else:
                paragraphs.append("__No_ __text__ __in__ __the__ __header__")
                temp1.append("__No_ __text__ __in__ __the__ __header__")
                Firm_Info.append(firm_name)
            if head==(n_headers-1):
                paragraphs.append(' '.join(str(e) for e in alltext[(pos2+1):]))
                temp1.append(' '.join(str(e) for e in alltext[(pos2+1):]))
                Firm_Info.append(firm_name)

    for head,para in enumerate(temp1):
        temp = [headers[head],para]
        headers_paragraphs.append(temp)


In [17]:
# Export headers to csv file - to be saved in the working library
df_headers = pd.DataFrame(Header_policies)
df_headers.insert(loc=0, column='firm', value=firms)
df_headers.to_csv("headers.csv")


In [18]:
#Extract headers and the corresponding paragraphs
headers_paragraphs_df = pd.DataFrame(headers_paragraphs,columns=("Headers","Paragraphs"))
headers_paragraphs_df['Firm_Info'] = Firm_Info
headers_paragraphs_df

,Headers,Paragraphs,Firm_Info
0,PRIVACY,"privacy standard motor products, inc (“smp”) w...",STANDARD MOTOR PRODUCTS
1,PRIVACY POLICY,standex takes your privacy seriously. please ...,STANDEX INTERNATIONAL
2,INTRODUCTION,standex international corporation and its worl...,STANDEX INTERNATIONAL
3,WHAT IS PERSONAL DATA?,personal data is any information that relates ...,STANDEX INTERNATIONAL
4,THE INFORMATION WE COLLECT,we collect several different types of informat...,STANDEX INTERNATIONAL
5,HOW WE USE THE INFORMATION WE COLLECT,we use the information that we collect or that...,STANDEX INTERNATIONAL
6,TO WHOM WE DISCLOSE INFORMATION AND WHY,"except as described within this policy, we wil...",STANDEX INTERNATIONAL
7,HOW TO ACCESS AND CONTROL THE INFORMATION,we respect your privacy rights and provide you...,STANDEX INTERNATIONAL
8,DATA SECURITY AND STORAGE,we take all reasonable steps to protect the in...,STANDEX INTERNATIONAL
9,DATA RETENTION,standex retains personal data and relevant inf...,STANDEX INTERNATIONAL


In [19]:
# Selecting Paragrraphs with only 100 or more characters.
len_list = [len(x) for x in headers_paragraphs_df['Paragraphs']]
out_index = [i for i,x in enumerate(len_list) if x<100]
headers_paragraphs_df.drop(headers_paragraphs_df.index[out_index],inplace=True)
headers_paragraphs_df = headers_paragraphs_df.reset_index(drop=True)
headers_paragraphs_df.to_csv("headers_paragraphs.csv")
headers_paragraphs_df

,Headers,Paragraphs,Firm_Info
0,PRIVACY,"privacy standard motor products, inc (“smp”) w...",STANDARD MOTOR PRODUCTS
1,PRIVACY POLICY,standex takes your privacy seriously. please ...,STANDEX INTERNATIONAL
2,INTRODUCTION,standex international corporation and its worl...,STANDEX INTERNATIONAL
3,WHAT IS PERSONAL DATA?,personal data is any information that relates ...,STANDEX INTERNATIONAL
4,THE INFORMATION WE COLLECT,we collect several different types of informat...,STANDEX INTERNATIONAL
5,HOW WE USE THE INFORMATION WE COLLECT,we use the information that we collect or that...,STANDEX INTERNATIONAL
6,TO WHOM WE DISCLOSE INFORMATION AND WHY,"except as described within this policy, we wil...",STANDEX INTERNATIONAL
7,HOW TO ACCESS AND CONTROL THE INFORMATION,we respect your privacy rights and provide you...,STANDEX INTERNATIONAL
8,DATA SECURITY AND STORAGE,we take all reasonable steps to protect the in...,STANDEX INTERNATIONAL
9,DATA RETENTION,standex retains personal data and relevant inf...,STANDEX INTERNATIONAL


In [20]:
#This will generate a csv file for output firms.
res = list(set(headers_paragraphs_df['Firm_Info'].tolist()))
with open('Output_firm.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in res:
        writer.writerow([val])

In [21]:
# Peparing data for creating text file.
File_Headers = headers_paragraphs_df['Headers'].tolist()
File_Paragraphs = headers_paragraphs_df['Paragraphs'].tolist()
ls = headers_paragraphs_df['Firm_Info'].tolist()
File_Firms = [x for i, x in enumerate(ls) if ls.index(x) == i]
temp1 = []
for firm in File_Firms:
    temp = []
    for item in range(0,len(File_Headers)):
            if firm in ls[item]:
                temp.append(item)
    temp1.append(temp)

In [22]:
#Text file to be saved in the working directory
thefile = open('test.txt', 'w')

for index,firm in enumerate(File_Firms):
    thefile.write("\n# %s \n" %firm)
    #thefile.write("\n--@@@@%s@@@@--\n" %urls[firm][0])
    for i in temp1[index]:
        try:
            thefile.write("\n## %s \n %s \n" % (File_Headers[i],File_Paragraphs[i]))
            print("Header: %s Success"%(File_Headers[i]))
        except Exception as e:
           
                print(e)
        
thefile.close()

Header: PRIVACY Success
Header: PRIVACY POLICY Success
Header: INTRODUCTION Success
Header: WHAT IS PERSONAL DATA? Success
Header: THE INFORMATION WE COLLECT Success
Header: HOW WE USE THE INFORMATION WE COLLECT Success
Header: TO WHOM WE DISCLOSE INFORMATION AND WHY Success
Header: HOW TO ACCESS AND CONTROL THE INFORMATION Success
Header: DATA SECURITY AND STORAGE Success
Header: DATA RETENTION Success
Header: OUR POLICY TOWARDS CHILDREN Success
Header: CALIFORNIA PRIVACY RIGHTS Success
Header: CHANGES TO OUR PRIVACY POLICY Success
Header: TERMS OF USE Success
Header: CONTACT INFORMATION Success
Header: SUBSCRIBE TO OUR NEWSLETTER Success
Header: NEWS & ANNOUNCEMENTS Success
Header: TRADESHOWS & EVENTS Success
Header: PRIVACY OVERVIEW Success
Header: STRICTLY NECESSARY COOKIES Success
Header: DATA PRIVACY STATEMENT Success
Header: HANDLING OF PERSONAL DATA Success
Header: ACCESSING OUR WEBSITE Success
Header: WHAT ARE COOKIES? Success
Header: WHAT COOKIES DO WE USE? Success
Header: OP

In [23]:
len(okurl)

105

In [24]:
#this function(tdm_df) would be used to extract words from the policies - eventually to create TDM matrix
from pandas import *
def tdm_df(doclist, stopwords = [], remove_punctuation = True, 
           remove_digits = True, sparse_df = False):
    
    # Create the TDM from the list of documents.
    tdm = txtm.TermDocumentMatrix()
  
    for doc in doclist:
        if remove_punctuation == True:
            doc = doc.translate(None, string.punctuation.translate(None, '"'))
        if remove_digits == True:
            doc = doc.translate(None, string.digits)
            
        tdm.add_doc(doc)
    
    # Push the TDM data to a list of lists,
    # then make that an ndarray, which then
    # becomes a DataFrame.
    tdm_rows = []
    for row in tdm.rows(cutoff = 1):
        tdm_rows.append(row)
        
    tdm_array = np.array(tdm_rows[1:])
    tdm_terms = tdm_rows[0]
    df = DataFrame(tdm_array, columns = tdm_terms)
    
    # Remove stopwords from the dataset, manually.
    # TermDocumentMatrix does not do this for us.
    if len(stopwords) > 0:
        for col in df:
            if col in stopwords:
                del df[col]
    
    if sparse_df == True:
        df.to_sparse(fill_value = 0)
    
    return df

In [25]:
#create a dictionary to store data frames for each url 

TDM = {}
for i in firms:
    TDM[i] = {}
    
TDM.keys()



dict_keys(['STANDARD MOTOR PRODUCTS ', 'STANDEX INTERNATIONAL ', 'BAYER AG ', 'DEUTSCHE LUFTHANSA AG ', 'AMCOR ', 'STATE STREET ', 'STEPAN ', 'VOLKSWAGEN AG ', 'NIELSEN HOLDINGS ', 'PUBLIC STORAGE ', 'STRYKER ', 'NAVIENT ', 'STURM RUGER ', 'NOVARTIS AG ', 'SUNTRUST BANKS ', 'SUPERIOR INDUSTRIES ', 'SYNALLOY ', 'SAPPI ', 'SYSCO ', 'CGG ', 'SAP SE ', 'TEJON RANCH ', 'TELEFLEX ', 'TENNECO ', 'TENNESSEE VALLEY AUTHORITY ', 'AMERICAN AIRLINES GROUP ', 'TERADYNE ', 'UNITED AIRLINES ', 'TEXAS INSTRUMENTS ', 'CECO ENVIRONMENTAL ', 'CEMEX SAB DE CV ', 'TEXTRON ', 'ARC RESOURCES ', 'THERMO FISHER SCIENTIFIC ', 'JAEGER RESOURCES ', 'AMERICAN TOWER ', 'THOR INDUSTRIES ', 'BADGER DAYLIGHTING ', 'TIDEWATER ', 'SIX FLAGS ENTERTAINMENT ', 'AFRICA OIL ', 'TIMKEN ', 'JAGUAR FINANCIAL ', 'CLARKE ', 'COLUMBIA SPORTSWEAR ', 'TELSON MINING ', 'CTD HOLDINGS ', 'APPLIED DNA SCIENCES ', 'TORCHMARK ', 'TORO ', 'LIGHTWAVE LOGIC ', 'TOTAL SYSTEM SERVICES ', 'F M BANK ', 'FORTUNE MINERALS ', 'FREEHOLD ROYALTIES ',

In [26]:
#extract unigrams for each firm 
#policy that is longest and meets the mimimum criteria is used for each firm

from IPython.display import display
pd.options.display.max_rows = None
 
for firm in firms:
    for index in okurl[firm]:
        TDM[firm][index] = {}
        TDM[firm][index] = tdm_df([scrape_policies[firm][index]],stopwords = rsw,remove_punctuation = False,remove_digits = False)


In [27]:
# Create a list to store unique firms
master_data=[]
selected_firms=[]

for firm in firms:
    index = okurl[firm]
    if len(index) != 0:
        master_data.append(scrape_policies[firm][okurl[firm][0]])
    #else:
      #  master_data.append('__aemptya__')
    
len(master_data)


105

In [28]:
#create term document matrix (TDM) using the master data

master_tdm = tdm_df(master_data,stopwords = rsw,remove_punctuation = False,remove_digits = False)
master_tdm.shape

(105, 8615)

In [29]:
#geographic indicator indicates if name(s) of any of the US state(s)
#appear in the document term matrix (TDM) for a given firm 
geography_information = []
geography_indicator = []

#names of the states have been appended - can be seen in the right most column of the output file
for index,firm_data in enumerate(master_data):
    temp = ""
    for state in states:
        if state in firm_data:
            temp = temp + ' ' + state
    if len(temp) != 0:
        geography_information.append(temp)
        geography_indicator.append(1)
    else:
        geography_information.append('__NO__')
        geography_indicator.append(0)

In [30]:
#assign geographic information to the TDM
master_tdm["geography_indicator"] = geography_indicator
master_tdm['geography_information'] = geography_information

In [31]:
#export unigram matrix to csv
master_tdm.to_csv("unigram.csv",sep=",")

In [32]:
#this will prepare data to create bigrams 
#the structure of the bigram file (BDM) will remain same as that of the TDM
def testFuncNew(text= "Hello, I am fine",rsw=rsw):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in rsw])
    return text
#store bigrams
list_data = []

#loop over each firm
for firm in firms:
    length = okurl[firm]
    breaks = firm.lower().split()
    for i in breaks:
        rsw.append(i)
    if len(length) != 0:
        tdm = txtm.TermDocumentMatrix()
        doc = scrape_policies[firm][okurl[firm][0]]
        tdm.add_doc(doc)

        tdm_rows = []
    
        for row in tdm.rows(cutoff = 1):
            tdm_rows.append(row)
        
        tdm_array = np.array(tdm_rows[1:])
        tdm_terms = tdm_rows[0]

        text = ""
        for index , word in enumerate(tdm_terms):
            text = text + " " + word 
        text = testFuncNew(text,rsw = rsw)    
        list_data.append(text)
    else:
        list_data.append("")
list_data

['personal information collected throughout privacy policy variations term indicate data smp affiliates identify include limited name title residential address mail company phone fax vehicle voluntary basis register enroll promotions sweepstakes submit purchase content join mailing list provide focus marketing campaigns furthermore reserves aspects generate user demographics administer computer systems troubleshoot website mobile application issues seek solicit confidential proprietary ideas suggestions materials consider communicate label includes feedback questions comments product applications correspondence submissions property shared sell rent exchange identifying unaffiliated third parties share related unrelated law requires addition due global nature locations store websites links party sites covered responsible view browsing neither mean endorses imply approved revisions sole discretion amend adapt time fit suggests periodically return page aware changes continued means accept

In [33]:
#use nltk to create consin dissimilarity matrix
#Function for eleminating stopwords and 

counter_sum = Counter()
for line in list_data:
    bigrams = list(nltk.bigrams(line.split()))
    bigramsC = Counter(bigrams)
    counter_sum += bigramsC
#export the bigram matrix to a csv file - to be saved in the working directory

with open('bigram.csv', 'w', newline='') as csvfile:
    header = sorted(counter_sum, key=lambda x: str(type(x)))
    writer = csv.DictWriter(csvfile, fieldnames=header)
    writer.writeheader()
    for line in list_data:
        bigrams = list(nltk.bigrams(line.split()))
        bigramsC = Counter(bigrams)
        cs = dict(counter_sum)
        bc = dict(bigramsC)
        row = {}
        for element in list(cs):
            if element in list(bc):
                row[element] = bc[element]
            else:
                row[element] = 0
        writer.writerow(row)

In [34]:
#compute cosine similarities 
#values are stored in a matrix - looks similar to a var-covar or correlation matrix

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df=1)
text = vect.fit_transform(list_data)

temp_array = (text * text.T).A


In [ ]:
#export cosine similarities matrix to a csv file
import pandas as pd
# 
df_cosine = pd.DataFrame(temp_array,columns=firms,index=firms)
df_cosine
df_cosine.to_csv("F:/Cosine.csv")

In [ ]:
# Make a worldcloud for the unigrams.
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Generate word cloud image
wordcloud = WordCloud().generate(text)


text = str(master_tdm)

wordcloud = WordCloud(max_font_size=40).generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
# save results to json # if does not work run the very first tab again(import the libraries again) and then run 
path = 'C:/Users/Lenovo/Desktop/NLP/'
with open(path + 'hmda_scraped_policies.json', 'w') as fp:
    json.dump(scrape_policies, fp)